In [ ]:
''' Creation: 2021.09.07
    Last update: 2021.09.12
    
    Useful functions for correctly parsing the aging data files
    '''

def data_parsing(df):
    index_data_header = [0]
    for ind,x in enumerate(df[' Voltage:    ']):
        if ('Volt' in x) or  ('Ref Cell' in x):
            index_data_header.append(ind)

    index_data_header.append(len(df))
    #print(index_data_header)
    list_df = []

    delta=0
    for i in range(len(index_data_header)-1):
        if i !=0 :
            delta=1
        list_df.append(df.iloc[df.index.isin(range(index_data_header[i]+delta,
                                                   index_data_header[i+1]))])
    return list_df

def drop_wrong_values(df):
    x = []
    y = []
    for data_x,data_y in zip(df[' Voltage:    '],df[' Current:']):
        try:
            data_x = float(data_x)
            if data_x > 0.0:
                x.append(data_x)
                y.append(float(data_y))
        except:
            pass
    return(x,y)

In [ ]:
''' Creation: 2021.09.12
    Last update: 2021.09.12
    
    User specific paths definition
    Based on information that should be available in the "pvu_user_config.json file" 
    '''

# Standard library imports
import sys
import os
import json
from pathlib import Path
from IPython.display import clear_output

# Local imports 
import PVcharacterization_Utils as pvu

## User identification
root = Path.home()

# Selection of folders of the PVcharacterization useful files
user_folders_list = [name for name in os.listdir(root) if os.path.isdir(os.path.join(root, name))]
user_folders_list = [name for name in user_folders_list if name[0] != '.']
user_folders_list.sort()
title = 'Select your configuration files folder (single selection)'
my_config_folder = root / Path(pvu.Select_items(user_folders_list,title,'single')[0])
clear_output(wait=True)
title = 'Select your PVcharacterization-aging files folder (single selection)'
my_pv_aging_files_path = root / Path(pvu.Select_items(user_folders_list,title,'single')[0])
clear_output(wait=True)

# Getting required information from user configuration file
file_config_users = my_config_folder / Path('pvu_user_config.json') 
with open(file_config_users, "r") as read_file:
    config_users = json.load(read_file)

## Add path of 'site-packages' where useful packages are stored on MAC-OS; no impact for Windows
#sys.path.append(config_users['mac_packages'])

# Paths identification for data access and results saving
my_pvaging_path = my_pv_aging_files_path / Path(config_users['agingdata_folder']+'/')
my_results_path = my_pv_aging_files_path / Path(config_users['agingresults_folder'] +'/')

# List of available bow data files
datafiles_list = os.listdir(my_pvaging_path)
datafiles_list = [name for name in datafiles_list if name[0] != '.']
datafiles_list.sort()

print('Specific paths set for ' + '"' + config_users['user'] + '"')
print('\nConfiguration path:', my_config_folder)
print('\nBow data')
print('Path:              ' , my_pvaging_path)
print('Files list:     \n',datafiles_list)
print('\nResults path:      ' , my_results_path)

In [ ]:
''' Creation: 2021.09.12
    Last update: 2021.09.12
    
    Selection by the user of the data files
    corresponding to the same irradiance
    '''

# Standard library imports
import os

# Local imports 
import PVcharacterization_Utils as pvu

title = 'Select the 3 aging rawdata files (T0, T1 and T2 )'
datafiles_select = pvu.Select_items(datafiles_list,title,'multiple')

dic = {}
times = []
for i in range(len(datafiles_select)):
    time = os.path.splitext(datafiles_select[i])[0][-2:]
    times.append(time)
    dic[time] = {}
    dic[time]['filename'] = os.path.splitext(datafiles_select[i])[0]
    
print(times)
print(dic)
#print('List of sorted aging times:',dic["aging_times_file"])

In [ ]:
''' Creation: 2021.09.07
    Last update: 2021.09.12
    
    For each file (.csv):
    - Read the header of the data file as meta_data dataframe and save as part of dict "dic"
    - Read the data located in the data file after the header, as dataframe
    - Parse the dataframe to locate the I(V) data by searching for specific column headers
    - Remove the lines where data are equal to '0.0'in each of the parsed dataframes
    - Convert the I(V) dataframe of interest into x,y data to plot
    
    Compute item variations for 3 couples of time : (T0,T1), (T0,T2) and (T1,T2)
    
    '''

# Standard library imports
import os
import sys
from pathlib import Path

# 3rd party imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
item_list = [('IrrCorr','W/m2'),('Pmax','W'),('Voc','V'),('Isc','A'),('Fill Factor','')]

for time in times:
    
    filepath = my_pvaging_path / Path(dic[time]['filename'] + '.csv')
    print('\n',filepath,'\n')

    # Read the metadata in the file (83 lines)
    header_length = 83
    if sys.platform == 'win32': 
        data_deb = header_length*2-1
    elif sys.platform == 'darwin':
        data_deb = header_length

    meta_data = pd.read_csv(filepath,
                     sep=',',
                     nrows=header_length,
                     header=None
                     ) # reads the meta data
    meta_data = dict(zip(meta_data[0],meta_data[1]))
    meta_data = {key.split(':')[0]:val for key,val  in meta_data.items()}
    
    assert time == meta_data['ID'][-2:],'Aging time in datafile' + dic[time]['filename']+' schould be '+ time
    
    print('Time:',time)    
    for item in item_list:
        dic[time][item[0]] = float(meta_data[item[0]])
        print(item[0]+':',dic[time][item[0]],item[1])
    
    # Read the I(V) data
    df_data = pd.read_csv(filepath,
                 sep=',',
                 skiprows=data_deb,
                 ) # reads the data   
    list_df_data = data_parsing(df_data)
    color =["r",'g','b']
    x,y = drop_wrong_values(list_df_data[0])
    plt.plot(x,y)
    /*plt.show()

# Compute item variations
t0 , t1 , t2 = times[0] , times[1] , times[2]
irr_key = item_list[0][0]
irradiance =  dic[t0][irr_key]
assert (dic[t1][irr_key] == irradiance and dic[t2][irr_key] == irradiance),\
'Irradiances of selected files should be the same'

print('Irradiance:',irradiance,item_list[0][1])
dic_var = {}
dic_var[irradiance]= {}
t_tuples = [(t0,t1),(t0,t2),(t1,t2)]
for t_tuple in t_tuples:
    aged = t_tuple[1]
    init = t_tuple[0]
    delta_t = aged + '-' + init
    print('\nVariations for',delta_t)
    dic_var[irradiance][delta_t] = {}
    for item in item_list[1:]:
        ditem_key = 'Delta '+ item[0]
        dic_var[irradiance][delta_t][ditem_key]= ((dic[aged][item[0]] - dic[init][item[0]])\
                                                              /dic[init][item[0]])*100    
        print(ditem_key + ':',round(float(dic_var[irradiance][delta_t][ditem_key]),4),'(%)')